In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df = pd.read_csv("..\Streamlit\data\compras_completo_limpio2.csv", low_memory=False)
df

In [ ]:
df["antiguedad"] = df["antiguedad"].replace("Sin especificar", "10-20")

In [ ]:
df["garaje"].unique()

In [ ]:
df["aire_acondicionado"].unique()

In [ ]:
df["conservacion"] = df["conservacion"].replace("Sin especificar", "En buen estado")

In [ ]:
df["latitud"] = df["coordenadas"].str.split(",").str[0].str.strip().astype(float, errors='ignore')
df["longitud"] = df["coordenadas"].str.split(",").str[1].str.strip().astype(float, errors='ignore')

In [ ]:
columnas_clustering = ["precio", "antiguedad", "conservacion", "latitud", "longitud", "piscina", "baños","habitaciones","superficie","superficie_util","aire_acondicionado", "provincia","planta","orientacion"]

In [ ]:
df = df.drop_duplicates(subset="identificador")
df

In [ ]:
df2 = df[columnas_clustering].copy()
df2

In [ ]:
df2 = df2.dropna(subset=['precio'])

In [ ]:
df2["orientacion"].unique()
label_encoder = LabelEncoder()
df2['orientacion'] = label_encoder.fit_transform(df2['orientacion'])

In [ ]:
label_encoder = LabelEncoder()
df2['antiguedad'] = label_encoder.fit_transform(df2['antiguedad'])


df2

In [ ]:
label_encoder = LabelEncoder()
df2['piscina'] = label_encoder.fit_transform(df2['piscina'])
df2

In [ ]:
label_encoder = LabelEncoder()
df2['planta'] = label_encoder.fit_transform(df2['planta'])
df2["planta"].unique()

In [ ]:
label_encoder = LabelEncoder()
df2['provincia'] = label_encoder.fit_transform(df2['provincia'])
df2

In [ ]:
label_encoder = LabelEncoder()
df2['conservacion'] = label_encoder.fit_transform(df2['conservacion'])
df2

In [ ]:
label_encoder = LabelEncoder()
df2['aire_acondicionado'] = label_encoder.fit_transform(df2['aire_acondicionado'])
df2

In [ ]:
df2 = df2.dropna()
df2

In [ ]:

X = df2.drop(columns=["precio"])  
y = df2["precio"] 
X["latitud"] = pd.to_numeric(X["latitud"], errors="coerce")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X.dtypes)


In [ ]:
model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"🔹 R²: {r2:.4f}")
print(f"🔹 RMSE: {rmse:.2f}")

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [4, 6, 8],
    "learning_rate": [0.01, 0.1, 0.2]
}

grid = GridSearchCV(xgb.XGBRegressor(objective="reg:squarederror"), param_grid, cv=3, scoring="r2")
grid.fit(X_train, y_train)

print("Mejores hiperparámetros:", grid.best_params_)

In [ ]:
best_model = xgb.XGBRegressor(
    objective="reg:squarederror",
    learning_rate=0.1,
    max_depth=8,
    n_estimators=300,
    random_state=42
)

# Entrenar el modelo
best_model.fit(X_train, y_train)

# Evaluar el nuevo modelo
y_pred_best = best_model.predict(X_test)
r2_best = r2_score(y_test, y_pred_best)
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))

print(f"🔹 R² Mejorado: {r2_best:.4f}")
print(f"🔹 RMSE Mejorado: {rmse_best:.2f}")